In [116]:
# -*- coding: utf-8 -*-

# Traitement des TRS pour ajouter les informations de BDLexique

In [117]:
import time
import sys
import codecs
import re
import pdb # ajouter pdb.set_trace() à l'endroit où on veut le débugueur
from lxml import etree as ET
import bs4
# import xml.etree.cElementTree as ET


In [118]:
import glob
import os, fnmatch

In [119]:
parser = ET.XMLParser(remove_blank_text=True)
debug=False

# Tests EAF

## Parser EAF

In [129]:
nomEAF="Test-Elan.eaf"
xmlEAF=ET.parse(nomEAF,parser)

In [130]:
ts={}
for timeOrder in xmlEAF.xpath("//TIME_ORDER/TIME_SLOT"):
    tsID=timeOrder.attrib["TIME_SLOT_ID"]
    tsTime=timeOrder.attrib["TIME_VALUE"]
    ts[tsID]=tsTime
# ts

In [131]:
def time2TRS(time):
    return unicode(float(time)/1000)

In [132]:
# time2TRS("25001")

In [137]:
tiersTypes={}
annotations={}
turnTextes=[]
for tier in xmlEAF.xpath("//TIER"):
    tierID=tier.attrib["TIER_ID"]
    tierType=tier.attrib["LINGUISTIC_TYPE_REF"]
    tiersTypes[tierID]=tierType
    for annotation in tier.xpath("ANNOTATION/ALIGNABLE_ANNOTATION"):
        aID=annotation.attrib["ANNOTATION_ID"]
        aTS1=annotation.attrib["TIME_SLOT_REF1"]
        aTS2=annotation.attrib["TIME_SLOT_REF2"]
        aBegin=time2TRS(ts[aTS1])
        aEnd=time2TRS(ts[aTS2])
        aValue=annotation.xpath("ANNOTATION_VALUE/text()")
        turnTextes.append("\n".join(aValue))
        turn=(tierID, aBegin, aEnd, aValue)
        annotations[aID]=turn

In [138]:
turnTextes

[u'comment \xe7a va ici ',
 'pas mal hein ',
 'super ',
 "alors t'vois",
 u'tu peux travailler ici comme \xe7a',
 'hmm',
 'bien tendre ta forme ',
 u"comme \xe7a de sorte qu'elle soit bien",
 u'de la m\xeame \xe9paisseur ',
 u"et \xe7a lisse un p'tit peu aussi tu vois ",
 '[inaudible]',
 u'comme \xe7a hop',
 "le haut c'est pareil",
 u"l'ext\xe9rieur tu peux aussi utiliser alors",
 'euh',
 u"alors tu mets ta main \xe0 l'int\xe9rieur pour pas que ton doigt se d\xe9forme",
 u"cet outil qu'on appelle une est\xe8que",
 u'tu vas un peu caresser mais tout en rentrant dans la mati\xe8re',
 u'tu vois la diff\xe9rence entre cette partie-l\xe0 et cette partie-l\xe0',
 u'l\xe0 le le grain peut commencer \xe0 rentrer ',
 u'et \xe7a lisse et tu vois \xe7a \xe7a enl\xe8ve les traces de doigts',
 u'\xe7a te fait une plus jolie forme qui est d\xe9j\xe0 chouette hein ',
 'ouais ',
 u"regarde y'en a un l\xe0 et sinon y'a le  smartphone l\xe0-bas ouais ",
 'comme tu veux fin le faux smartphone ',
 u"avec 

In [146]:
def trimer(mot):
    mot=mot.lower()
    for p in u',;.-?!“”‘’‛‟′″´˝"«»':      # Modifié le 12/01/20 pour gérer les deux points comme marqueur dans les mots
        mot=mot.replace(p, ' ')
    mot=mot.strip()
    return mot

def listerMots(texte):
    listeMots={}
    elements=re.findall(ur"[\wâàéèêëîïôùûüçÂÀÉÈÊËÎÏÔÙÛÜÇæœÆŒ():]+|[-.…,—–()\[\]\/#\"“”‘«»<>'’=~]| ?[;!?:]", texte)
    elements=[x for x in elements if x!=u" "]
    mots=[x for x in elements if not x in u"-_.…,—–()\[\]\/#\"“”‘«»<>'’=~:" and not x in [u" ;",u" !",u" ?",u" :"]]
    for mot in mots:
        mot = trimer(mot)
        (forme,graphie)=deparentheser(mot)
        listeMots[forme]=graphie
    return listeMots

In [149]:
texte="\n".join(turnTextes)
listeMots=listerMots(texte)
for mot in listeMots:
    print mot, listeMots[mot]

 attrape attrape
justement justement
rentres rentres
rentrer rentrer
rallonge rallonge
dessus dessus
déjà déjà
pique pique
per per
chair chair
veux veux
enfoncer enfoncer
sympa sympa
inverse inverse
êtes êtes
endroit endroit
pouvoir pouvoir
donner donner
tait tait
tu tu
vez vez
rouleau rouleau
percer percer
aut aut
génial génial
nord nord
suite suite
te te
idée idée
sent sent
ta ta
pens pens
vers vers
smartphone smartphone
trompe trompe
prends prends
grenouille grenouille
disais disais
trous trous
flora flora
sont sont
faudrait faudrait
garder garder
voit voit
ohlala ohlala
voir voir
vois vois
cool cool
veu veu
apprendre apprendre
sans sans
vu vu
durcir durcir
poitrine poitrine
rires rires
large large
p p
dirait dirait
dix dix
fortes fortes
dit dit
monologue monologue
dis dis
coussin coussin
compliqué compliqué
intelligente intelligente
osé osé
redessiner redessiner
droit droit
zappé zappé
aux aux
tes tes
pris pris
tournée tournée
font font
circonflexe circonflexe
filles filles
élément

## Construire TRS

### Entête Trans
- définir la racine Trans
    - associer la liste des Speakers
    - associer un épisode avec une section

In [126]:
root=ET.Element("Trans")

speakers=ET.SubElement(root, "Speakers")
speakerID={}
for nSpk,spk in enumerate(tiersTypes):
    ET.SubElement(speakers,"Speaker",id="spk%d"%(nSpk+1),name=spk)
    speakerID[spk]="spk%d"%(nSpk+1)

episode=ET.SubElement(root, "Episode")
section=ET.SubElement(episode, "Section")

### Ajouter les Turns
- suivre l'ordre des interventions de Elan => sorted
    - ajouter la balise Turn
        - speaker
        - startTime
        - endTime
    - ajouter le contenu du tour
        - structurer les tours avec des balises internes [balise]

In [127]:
for k,v in sorted(annotations.iteritems(),key=lambda x: int(x[0].strip("a"))):
    #
    # créer la structure Turn avec speaker, startTime, endTime
    #
    turn=ET.SubElement(section,"Turn",speaker=speakerID[v[0]],startTime=v[1],\
                                      endTime=v[2])
    #
    # découper le texte de l'annotation
    #
    vText="\n".join(v[3])
    vParts=re.split(ur"\[(.*?)\]",vText)
    #
    # si le texte n'a qu'un morceau, l'insérer dans Turn
    #
    if len(vParts)==1:
        turn.text=vText
    #
    # si le texte a plusieurs morceaux, 
    #   insérer le premier morceaux
    #   associer des balises (morceaux impairs)
    #   insérer les autes morceaux pairs comme tail de la balise précédente
    #
    else:
        for nV,vPart in enumerate(vParts):
            if nV==0:
                turn.text=vPart
            elif nV%2==1:
                baliseInterne=ET.SubElement(turn,vPart.replace(" ","_"))
            else:
                baliseInterne.tail=vPart
##########
# ajouter un traitement des balises à contenu
##########

In [128]:
tree = ET.ElementTree(root)
tree.write("TEMP.xml", xml_declaration=True, encoding='utf-8')

## MODIFICATIONS FAITES :
1. changer l'organisation pour pouvoir traiter une liste de dossiers plutôt que un dossier
  - garder le lexique général en entrée
  - changer les sorties pour avoir un jeu d'exceptions par dossier
1. ajouter un #id aux tours et aux mots => **22/12/15**
1. gérer les parenthèses => **22/12/15**
  - les troncations
    - version longue pour BDLexique
    - version courte pour la transcription
  - les champs supplémentaires
    - mot => nbsyllabes à saisir
    - tour => raccourci
1. gérer les balises Event auto-fermantes => **23/12/15**
1. gérer les connecteurs
  - trouver les connecteurs multi-mots
  - tour => liste de connecteurs

# Préparation de l'environnement pour le script
- *dossier* doit être le répertoire où se trouvent vos fichiers (devrait finir par un /)
- *fichierTRS* contient la liste des noms de vos fichiers TRS à traiter (rempli automatiquement)
- *fichierLexique* doit être le nom du fichier BDLEXIQUE
- *fichierExceptions* doit être le nom de votre fichier INCONNUS


In [5]:
altEspace=ur"\s|_"
connecteurs=[
    u"et", u"alors", u"du coup", u"sinon", u"par contre", u"ça veut dire", u"enfin",
u"après", u"donc", u"puisque", u"puisqu'", u"en fait", u"mais", u"parce que", u"parce qu'", u"même si" , u"d'abord", u"et puis"
]
immigration=[u"cible",u"frontières",u"accueillir",u"démonstrations",u"accompagner",u"contre-courant",u"importantes",u"droit de bouger",u"peur",u"solution",u"durable",u"refuge",u"assassins",u"tueurs",u"violeurs",u"fuient",u"misère",u"chaos",u"circulation",u"liberté",u"installation",u"solidarité",u"migrants",u"ouverture",u"frontières",u"conditions",u"réflexe",u"normal",u"sain",u"accueillir",u"moyens",u"intégrés",u"flux",u"migratoires",u"énormes",u"moyens",u"structures",u"loger",u"répondre",u"peuples",u"coopération",u"indéniable",u"réfugiés",u"digne",u"prendre la main",u"équilibrée",u"accueil",u"correctement",u"favorable",u"hébergés",u"mise à l’abri",u"centre humanitaire",u"situation",u"convenable",u"heurtée",u"valeurs",u"république",u"comportement",u"politique",u"associations",u"hébergement",u"respect",u"lien",u"confiance",u"travailleur",u"social",u"policiers",u"gazent",u"détruisent",u"nourriture",u"refusent",u"points d’eau",u"négation",u"partent",u"parcours",u"migration",u"terribles",u"touche",u"cimetière",u"choquant",u"scandaleux",u"dignité",u"impression",u"responsabilités",u"multiplication",u"conflits",u"excessivement",u"violents",u"solution",u"seule",u"quitter",u"victimes",u"esclavage",u"regression",u"humanité",u"guerre",u"état",u"pauvreté",u"pays",u"conditions",u"respect",u"organisme",u"ficher",u"contrôler",u"demande",u"papiers",u"droits",u"échec",u"aide",u"développement",u"vivre",u"famille",u"terrain",u"voir",u"souffrance",u"regression",u"France",u"richesse",u"humanité",u"extraordinaire",u"intérger",u"asile",u"question",u"temporaire",u"débat",u"souhaitons",u"propositions",u"européen",u"rapprocher",u"proches",u"position",u"absurdité",u"ONG",u"aider",u"détresse",u"lois",u"règles",u"principes",u"humanité",u"vulnérable",u"enfants",u"sauver",u"vies",u"générosité",u"classes",u"francophones",u"découverte",u"langue",u"logements",u"associations",u"continuerai",u"nouveaux",u"clé",u"chômage",u"héberger",u"employer",u"former",u"éduquer",u"milliers",u"vaincre",u"réticences",u"acccepter",u"aménage",u"refuges",u"régions",u"viennent",u"zones",u"protégées",u"normales",u"chez-eux",u"circuler",u"librement",u"Allemagne",u"obligations",u"humanitaire",u"préserver",u"populations",u"bataille",u"hégémonique",u"bataille",u"s'installer",u"mois",u"désaccord",u"moins",u"accueillant",u"bataille",u"idéologique",u"pression",u"chercher",u"immigrés",u"de force",u"délogés",u"expulsés",u"xénophobie",u"racistes",u"recul",u"idées",u"argent",u"dramatique",u"problèmes",u"économiques",u"interventions",u"militaires",u"famine",u"massacres",u"répression",u"bombardé",u"manqué",u"devoir",u"indigne",u"déposer",u"demande d'asile",u"absurde",u"campements",u"regrette",u"dispositifs",u"pérennes",u"divergence",u"appréciation",u"reconstitution",u"campements",u"crise",u"emballement",u"otage",u"noyer",u"guerre",u"bombardement",u"morts",u"difficultés",u"blocage",u"maltraiter",u"seul",u"privation",u"absurde",u"illusion",u"mensonge",u"ghettos",u"français",u"divisés",u"meurent",u"fuient",u"devoirs",u"moraux",u"centres de rétention",u"protection",u"cause",u"guerre",u"lieu",u"couteux",u"charge",u"services",u"gratuits",u"système",u"gestion",u"patrimoine",u"nationale",u"importante",u"fossé",u"travail",u"difficulté",u"maîtrise",u"suspendre",u"shengen",u"arrêter",u"immigration",u"communautarisme",u"fondamentalisme",u"islamiste",u"gel",u"constructions",u"mosquées",u"islam",u"radicaux",u"vivier",u"radicalisation",u"soldats",u"djihadisme",u"terroristes",u"puissance",u"terreau",u"frappent",u"clandestine",u"massive",u"insécurité",u"terrifiante",u"forces de l'ordre",u"mesures législatives",u"suppression",u"enrayer",u"processus",u"récupérer",u"clandestins",u"absence",u"bateaux",u"en mer",u"ramener",u"facilite",u"réseaux",u"criminels",u"laisser",u"mourir",u"sauver",u"ramener",u"port d’origine",u"corrompus",u"passeurs",u"arme",u"catastrophique",u"occupation",u"illégale",u"inexpulsable",u"engrenage",u"procédures",u"demandeurs",u"déboutés",u"économique",u"raisons",u"traverser",u"sortis",u"territoire",u"arrivés",u"origine",u"critères",u"extrêmement",u"incitatif",u"centres",u"AME",u"CMU",u"ensemble",u"folle",u"coût",u"faramineux",u"malvenu",u"opposition",u"opposée",u"communautarisme",u"fondamentalisme",u"imposer",u"ennemi",u"contre",u"rejet",u"anti-démocratique",u"enfermemement",u"séparation",u"pourissement",u"étranger",u"drame",u"faillite",u"déloyale",u"ménage"
]

In [6]:
immigrationMots=sorted(set(immigration))

In [7]:
update=False
#dossierCorpus="/Users/gilles/Downloads/LNS3U5/"
dossierCorpus="/Users/gilles/pCloud Drive/FOD/Corpus-2017/0 Groupes/"
#dossierCorpus="/Users/gilles/pCloud Drive/FOD/Corpus-2017/0AjustementsTIC/"
dossierCorpus="/Users/gilles/pCloud Drive/FOD/Corpus-2018/0 Groupes/"
dossierCorpus="/Users/gilles/pCloud Drive/FOD/GB/Corpus-2019/0 Groupes/"
dossierCorpus="/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0 Groupes/"
fichierXSLT="/Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS-phon.xsl"
corpusDossiers=glob.glob(dossierCorpus+"*/")
dossiersTRS={dCorpus:dCorpus+"inconnus.txt" for dCorpus in corpusDossiers}
sansRebalisageFichiers=[
                        "enregistrement_2_G_A_D.trs",
                        "enregistrement_5.1_T_M-BDL2.trs",
                        "enregistrement_5.2_T_M.trs"
#                        "Chantal Véronique Giacomo.trs",
#                        "Les Filles à la fac 2211.trs",
#                        "Lili et Axelle au mcdo-transcription.trs",
#                        "heartstone.trs",
#                        "KirchnerovaAneta.trs",
#                        "Xeleko-4.3.trs",
#                        "KevinAzaisCesarMeilleurEspoirMasculin.trs",
#                        "Donald Trump Talks Media Coverage, Polls and His Vocal Transformation.trs",
#                        "Nicolas Sarkozy et questions libres.trs",
#                        "les filles à la fac 22nov.trs",
#                        "Linguistique (1) (mp3cut.net) (1).trs",
#                        "trs_3.trs",
#                        "trs_8.trs",
#                        "3Cours_Latin 5e _Jean-Pierre_Anne.trs"
                        ]
corpusDossiers

['/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0 Groupes/Solsona/',
 '/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0 Groupes/CordetteLefevreNaprix/',
 '/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0 Groupes/BadiGrousseau/',
 '/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0 Groupes/DupuyOlengaOwanlele/']

In [8]:
listeDossiersTRS=corpusDossiers
if update:
    updatedDossiers=glob.glob(dossierCorpus.replace("0 Groupes/","")+"[a-zA-Z]*/")
    filtreDossiers=[n.split("/")[-2] for n in updatedDossiers]
    filtreDossiers

    listeDossiersTRS=[c for c in corpusDossiers if c.split("/")[-2] in filtreDossiers and "Daganaud" in c]
listeDossiersTRS

['/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0 Groupes/Solsona/',
 '/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0 Groupes/CordetteLefevreNaprix/',
 '/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0 Groupes/BadiGrousseau/',
 '/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0 Groupes/DupuyOlengaOwanlele/']

In [9]:
numPremierDossier=0
numDernierDossier=0
if numDernierDossier<numPremierDossier or numDernierDossier==0:
    numDernierDossier=100
elif numDernierDossier==numPremierDossier:
    numDernierDossier+=1
numDossiers=range(numPremierDossier,numDernierDossier)

In [10]:
fichierLexique="/Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/bdlexique.txt"
fichier_exceptions=True

In [11]:
voyelles=u"ieɛayøœəuoɔɑɛ̃ɔ̃ɑ̃"
voyelles=u"ieEay296@uoOòèâêûô"

Si vous n'avez pas de fichier *inconnus.txt* 
>mettez *fichier_exceptions=False* au dessus

#Modif GB 12/04/14
- mise en texte des deux blocs de traitement de la ligne de commande

In [12]:
lexicon=codecs.open(fichierLexique,"r",encoding='utf8')
bdlexique=lexicon.readlines()
lexicon.close()

In [13]:
facultatives = 0

In [14]:
phon={}
result=[]
nouvellesExceptions = []
output=[]

### Préparation des fichiers

algorithme

ajouter chaque ligne du fichier à phrases[]

In [15]:
def lowerAccents(chaine):
    return chaine.lower()

### Normaliser le mot en cours

algorithme

+ la ponctuation est remplacée par un espace
+ les espaces aux extrémités sont effacés
+ le mot est mis en minuscules

In [143]:
def trimer(mot):
    mot=lowerAccents(mot)
#    for p in u',;.:-?!()“”‘’‛‟′″´˝"«»':
#     for p in u',;.:-?!“”‘’‛‟′″´˝"«»':   # Modifié le 22/12/15 pour gérer les parenthèses comme marqueur dans les mots
    for p in u',;.-?!“”‘’‛‟′″´˝"«»':      # Modifié le 12/01/20 pour gérer les deux points comme marqueur dans les mots
        mot=mot.replace(p, ' ')
    mot=mot.strip()
    return mot

In [144]:
def deparentheser(mot):
    forme=mot
    graphie=mot
    m=re.search(ur"\([^)]*\)",forme)
    while (m):
        forme=re.sub(ur"\(([^)]*)\)","\g<1>", forme)
        graphie=re.sub(ur"\(([\wâàéèêëîïôùûüçÂÀÉÈÊËÎÏÔÙÛÜÇæœÆŒ]+['’]?)\)","'", graphie)
        m=re.search(ur"\([^)]*\)",forme)
    forme=forme.lstrip(")").rstrip("(")
    forme=forme.replace(":","")
    if graphie.endswith("("):
        graphie=graphie[:-1]+"'"
    return(forme,graphie)
#    motsAbreges[mot]={"lexical":forme, "graphie":graphie}    
    

In [150]:
deparentheser(u"th(é)ra::peutiq(ue)")

(u'th\xe9rapeutique', u"th'ra::peutiq'")

In [19]:
def listerMotsCorpus(rootTRS):
    phrases=[]
    motsPhrases=[]
    elementsPhrases=[]
    motsCorpus=set()
    nPhrases=0
    for ligne in rootTRS.xpath("//Turn//text()"):
        line=ligne.strip()
        if 0: print [line]
    #    print nPhrases, line
        phrases.append(line)
#        elements=re.findall(ur"[\wâàéèêëîïôùûüçÂÀÉÈÊËÎÏÔÙÛÜÇæœÆŒ]+['’]?|[-.…,—–()\[\]\/#\"“”‘«»<>'’=~]| ?[;!?:]", line)
        if "_" in line:
            altLine=re.sub(altEspace," ",line)
            print altLine
        else: altLine=line
#         elements=re.findall(ur"[\wâàéèêëîïôùûüçÂÀÉÈÊËÎÏÔÙÛÜÇæœÆŒ()]+['’]?|[-.…,—–()\[\]\/#\"“”‘«»<>'’=~]| ?[;!?:]", altLine)
#         elements=re.findall(ur"[\wâàéèêëîïôùûüçÂÀÉÈÊËÎÏÔÙÛÜÇæœÆŒ()]+|[-.…,—–()\[\]\/#\"“”‘«»<>'’=~]| ?[;!?:]", altLine)
        # 
        # modif pour permettre les : pour indiquer la longueur des voyelles
        #
        elements=re.findall(ur"[\wâàéèêëîïôùûüçÂÀÉÈÊËÎÏÔÙÛÜÇæœÆŒ():]+|[-.…,—–()\[\]\/#\"“”‘«»<>'’=~]| ?[;!?:]", altLine)
#         print elements
        mots=[x for x in elements if not x in u"-_.…,—–()\[\]\/#\"“”‘«»<>'’=~:" and not x in [u" ;",u" !",u" ?",u" :"]]
#         print mots
        elements=[x for x in elements if x!=u" "]
        elementsPhrases.append(elements)
        phrasePropre = u""
        for mot in mots:
            mot = trimer(mot)
            phrasePropre += mot+u" "
            (forme,graphie)=deparentheser(mot) # modifié pour les :
#            m=re.search(ur"\(.*\)",mot)
#            forme=mot
#            graphie=mot
#            if m :
#                forme=re.sub(ur"\((.*)\)","\g<1>", mot)
#                graphie=re.sub(ur"\(([\wâàéèêëîïôùûüçÂÀÉÈÊËÎÏÔÙÛÜÇæœÆŒ]+['’]?)\)","'", mot)
#                motsAbreges[mot]={"lexical":forme, "graphie":graphie}    
            motsAbreges[mot]={"lexical":forme, "graphie":graphie}
            motsCorpus.add(forme)
#        phraseMots = phrasePropre.strip()
        phraseMots = phrasePropre.split()
#         print phraseMots
        motsPhrases.append(phraseMots)
        nPhrases+=1
    return (motsCorpus,motsPhrases,elementsPhrases)

algorithme

extraire de BDLex 0.forme fléchie, 1.phonétique, 2.liaison, 3.cat-gram, 4.genre+nombre

algorithme

extraire du fichier d'exceptions les mêmes données que pour BDLex

#Modif GB 12/04/14
- fait une liste des exceptions lues pour ne pas les rajouter à la fin
- éviter de tenir compte des exceptions non renseignées
 - les mots du fichier exceptions sans transcriptions étaient transcrits par une chaine vide...

## Fonctions

### Vérifier si le mot existe

algorithme

+ si le mot est dans BDLex, ok
+ s'il y a un espace dans le mot,
    * le mot est divisé en deux et
    * si les mots existent dans bdlex, ok
    * sinon les mots sont ajoutés aux nouvelles exceptions et mis entre étoiles
+ s'il y a un apostrophe dans le mot,
    * le mot est divisé en deux
    * si les mots existent dans bdlex, ok
    * sinon les mots sont ajoutés aux nouvelles exceptions et mis entre étoiles
+ dans les autres cas, le mot est ajouté aux nouvelles exceptions et mis entre étoiles

In [20]:
def verifier_mot(mot):
        sampa=""
        if mot in phon.keys():
            sampa += phon[mot][0]
        elif " " in mot:
            mots = mot.split()
            for mot in mots:
                if mot in phon.keys():
                    sampa += phon[mot][0]+" "
                elif mot != "":
                    nouvellesExceptions.append(mot)
                    sampa += "***"+mot+"*** "
        elif "'" in mot:
            mots = mot.split("'")
            mots[0]=mots[0]+"'"
            for mot in mots:
                if mot in phon.keys():
                    sampa += phon[mot][0]+" "
                elif mot != "":
                    nouvellesExceptions.append(mot)
                    sampa += "***"+mot+"*** "
        elif mot != "": 
            nouvellesExceptions.append(mot)
            sampa="***"+mot+"*** "
        return sampa

### 2. traduire le SAMPA de BDLexique en API

#Modif GB 12/04/14
- ajout du r et du â
- ajout des exemples associés en dessous

In [21]:
# traduire SAMPA-BDLex en API

def sampa2api(sampa):
    if isinstance(sampa,str):
        api=sampa.decode("utf8")
    else:
        api=sampa
    api=api.replace(u'n"',u'n') 
    api=api.replace(u't"',u't') 
    api=api.replace(u'z"',u'z') 
    api=api.replace(u'R"',u'ʁ') 
    api=api.replace(u'p"',u'p') 
    api=api.replace(u'S',u'ʃ') 
    api=api.replace(u'Z',u'ʒ')
    api=api.replace(u'N',u'ŋ')
    api=api.replace(u'J',u'ɲ')
    api=api.replace(u'r',u'ʁ') 
    api=api.replace(u'H',u'ɥ')
    api=api.replace(u'E',u'ɛ')
    api=api.replace(u'2',u'ø')
    api=api.replace(u'9',u'œ')
    api=api.replace(u'6',u'ə')
    api=api.replace(u'O',u'ɔ')
    api=api.replace(u'è',u'e')   
    api=api.replace(u'ò',u'o')    
    api=api.replace(u'â',u'ɑ̃')   
    api=api.replace(u'ê',u'ɛ̃')   
    api=api.replace(u'û',u'œ̃')  
    api=api.replace(u'ô',u'ɔ̃')       
    api=api.replace(u'@',u'ə')
    api=api.replace(u'R',u'ʁ') 
    return api

### 4. Vérifier si la liaison est possible

algorithme

+ si le mot courant et le suivant ne sont pas dans lexicon, pas de liaison
+ si le mot a une consonne dans le champ de la voyelle de liaison, check1 est vrai
+ si le mot suivant commence par une voyelle, check2 est vrai

  si check1 et check2 sont vrais, il y a liaison

In [22]:
def liaison_possible(phrase ,mot , mot_numero):
    check1=0
    check2=0
    if mot in phon and len(phrase)>mot_numero+1 and phrase[mot_numero+1] in phon:
        consonnes=['k"', '(kt)"', 'n"', 'p"', 'R"', '@t"', 't"', '-V', '+V', '@z"', 'z"']
        phoneme=phon[mot][2]
        for phoneme in consonnes:
            check1=1
        
        voyelles=["H", "j", "w", "E", "a", "2", "9", "6", "@", "y", "u", "O", u"ò", "o", "e", u"è", u"ê", u"û", u"ô", "i"]
        mot_suivant=phon[phrase[mot_numero+1]][1]
        for v in voyelles:
            if mot_suivant.startswith(v):
                check2=1

    if check1 and check2 :
        return True
    else:
        return False

### 5. vérifier si la liaison est obligatoire

algorithme

+ si le mot courant et le suivant sont dans un des cas de figure, il y a liaison
+ sinon pas de liaison

In [23]:
def liaison_obligatoire(phrase, mot, mot_numero):
    determinant=["d", "P"]
    nom=["N", "G", "M"]
    adjectif=["J", "G", "M"]
    pronompers=["P"]
    verbe=["V"]
    catgram_mot1=phon[phrase[mot_numero]][3]
    catgram_mot2=phon[phrase[mot_numero+1]][3]

    if catgram_mot1 in determinant and catgram_mot2 in nom :
        return True

    elif catgram_mot1 in determinant and catgram_mot2 in adjectif :
        return True
 
    elif catgram_mot1 in pronompers and catgram_mot2 in verbe :
        return True

    elif catgram_mot1 in verbe and catgram_mot2 in pronompers :
        return True

    else:
        return False

Cas de figure possibles:

- DET + N
    * ri + N:   d'animal, 
    * di + N:   certains éléphants
    * rd + N:   les animaux
    * dd + N:   ces étés, cet été
    * dp + N:   ton anorak
    * rc + N:   aux armes
- DET + ADJ:
    * ri + ADJ:   d'énormes
    * di + ADJ:   plusieurs immenses
    * rd + ADJ:   les immenses
    * dd + ADJ:   cet immense
    * dp + ADJ:   son immense
    * rc + ADJ:   aux immenses
- PERS + V:
    * SS + V:   m'épate
- V + PRO PERS: 
    * V + SS:   vont-ils


algorithme

+ si le mot courant et le suivant sont dans un des cas de figure, il y a liaison
+ sinon pas de liaison

In [24]:
# vérifier si la liaison est facultative
def liaison_facultative(phrase, mot, mot_numero):
    #pdb.set_trace()
    nom=["N", "G", "M"]
    pluriel=["MP", "FP"]
    adjectif=["J", "G", "M"]
    verbe=["V"]
    pronompers=["P"]
    adverbe=["A"]
    preposition=["p"]
    catgram_mot1=phon[phrase[mot_numero]][3]
    catgram_mot2=phon[phrase[mot_numero+1]][3]
    genre_mot1=phon[phrase[mot_numero]][4]
    
    if (catgram_mot1 in nom) and (phon[phrase[mot_numero]][4] in pluriel) and (catgram_mot2 in adjectif) : 
        return True

    elif (catgram_mot1 in verbe) and (catgram_mot2 not in pronompers):
        return True

    elif catgram_mot1 in adverbe :
        return True
    
    elif catgram_mot1 in preposition : 
        return True

    else :
        return False

Cas de figure possibles :

- N pl + ADJ: 
    * N + ADJ: monstres énormes 
    * G + ADJ: rivaux énormes
- VERBE + TOUT-SAUF-PRO-PERS:
    * V + N sont éléphants
    * V + G sommes abdicaires
    * V + V sommes assis
    * V + A sommes admirablement
    * V + p sommes autour de
    * V + di ont aucune
    * V + rc sommes au
- ADV + QQCH:
    * ADV + N vraiment abruti
    * ADV + G vraiment abandonné
    * ADV + V vraiment aimé
    * ADV + J vraiment étonnant
    * ADV + ss vraiment ils
    * ADV + A vraiment étonnamment
    * ADV + p vraiment attendu
    * ADV + di vraiment autre 
    * ADV + rc vraiment au
- PREP + QQCH:
    * PREP + N très amoureux
    * PREP + G très abandonné
    * PREP + V très aimé
    * PREP + J très étonnant
    * PREP + SS très ils
    * PREP + A très étonnamment
    * PREP + p très attendu
    * PREP + di très autre
    * PREP + rc très au


## Traitement

+ Partie 1
*chaque phrase est prise individuellement,
    * découpée en blocs,
        * qui sont chacuns trimés si ce sont des mots
        * s'il y a plusieurs mots dans le bloc, ils sont séparés
    + Partie 2
    * pour chaque couple de mots
        * si la liaison est possible,
            * et qu'elle est obligatoire, l'api avec la liaison est généré
            * et qu'elle est facultative,
                * si l'utilisateur l'a choisi, l'api avec la liaison est généré
                * sinon l'api sans la liaison est généré

        + Partie 3
        * si la liaison n'est pas possible,
            * si le mot est dans bdlex, l'api est généré
            * sinon le mot est laissé tel quel (il a déjà les étoiles)        

    * pour le dernier mot de la phrase, 
        * si le mot est dans bdlex, l'api est généré
        * sinon le mot est laissé tel quel (il a déjà les étoiles) 

+ Partie 4
* le message à l'utilisateur et la phrase en api est imprimée

#Modif GB 12/04/14
- suppression du délai dans la boucle
 - pour 1500 lignes => 3 secondes sans ralentisseur, 1503 secondes avec 

In [25]:
from lxml.builder import E

In [26]:
def compterVoyelles(chaine):
    result=0
    for element in chaine:
        if element in voyelles:
            result+=1
    return result

#Début de l'enchassement en XML (7/12/15)
- récupérer la ponctuation et les sauts de lignes pour rendre le texte lisible
- ajouter le reste des informations du lexique dans la balise

In [27]:
def enchasseBDLexique(nphrase,nmot,liaison=False):
    boolAbrege=False
    motTRS=motsPhrases[nphrase][nmot]
    if motTRS in motsAbreges:
        mot=motsAbreges[motTRS]["lexical"]
        graphie=motsAbreges[motTRS]["graphie"]
        boolAbrege=True
    else:
        mot=motTRS
        graphie=motTRS
#    print mot
    if mot in phon: 
        phono=sampa2api(phon[mot][1])
        if liaison:
            phono+=sampa2api(phon[mot][2])
        cat=phon[mot][3]
        if cat in [u"J",u"K"]:
            cat=u"Adj"
        ms=phon[mot][4]
        vs=phon[mot][5]
        lexeme=phon[mot][6].upper()
        freq=phon[mot][8]
        nbVoyelles=str(compterVoyelles(phon[mot][1]))
        if u" " in vs:
            vs=u""
    else:
        phono=verifier_mot(mot)[:-1]
        cat=u"???"
        ms=""
        vs=""
        lexeme="???"
        freq=""
        nbVoyelles=""
    motAttributs={"cat":cat,"ms":ms,"vs":vs,"phon":phono,"nbsyll":nbVoyelles, "lexeme":lexeme, "freq":freq, "id":"%05d%03d"%(nphrase,nmot)}
    if boolAbrege:
        motAttributs["ABnbsyll"]=""
        motAttributs["ABphon"]=""
    result=E.motBDL(graphie,motAttributs)
#    print etree.tostring(result,encoding="utf8")
#    print (cat,ms,vs,phono,mot)
#    u'<mot cat="%s" ms="%s" vs="%s" phon="%s">%s</mot>' % (cat,ms,vs,phono,mot)
    return result
    
def enchasseXML(mot, phono):
    if isinstance(phono,str):
        phono=phono.decode("utf8")
    result=E.motBDL(mot,{"phon":phono})
#    u'<mot phon="%s">%s</mot>' % (phono, mot)
    return result

def enchasseTour(phrase):
    result=E.tour(phrase,{"id":"%06d"%nPhrase})
    return result

def enchasseNonMot(nonmot):
    result=E.punct(nonmot)
#    u'<punct>%s</punct>' % (nonmot)
    return result

In [28]:
def traitementTRS(rootTRS):
    a=1
    nPhrase=0
    for ligne in rootTRS.xpath("//Turn//text()"):
        phrase=ligne.strip()
        api=E.tour()
        mot_numero=0
        element_numero=0
        while elementsPhrases[nPhrase] and element_numero < len(elementsPhrases[nPhrase]):
            if not mot_numero < len(motsPhrases[nPhrase]) or motsPhrases[nPhrase][mot_numero]!=elementsPhrases[nPhrase][element_numero].lower():
                api.append(enchasseNonMot(elementsPhrases[nPhrase][element_numero]))
            elif liaison_possible(motsPhrases[nPhrase], motsPhrases[nPhrase][mot_numero], mot_numero):
                if liaison_obligatoire(motsPhrases[nPhrase], motsPhrases[nPhrase][mot_numero], mot_numero):
                    api.append(enchasseBDLexique(nPhrase,mot_numero,True))
                elif liaison_facultative(motsPhrases[nPhrase], motsPhrases[nPhrase][mot_numero], mot_numero):
                    if facultatives:
                        api.append(enchasseBDLexique(nPhrase,mot_numero,True))
                    else :
                        api.append(enchasseBDLexique(nPhrase,mot_numero))
                else:
                    api.append(enchasseBDLexique(nPhrase,mot_numero))
                mot_numero+=1
            else:
                api.append(enchasseBDLexique(nPhrase,mot_numero))
                mot_numero = mot_numero+1
            element_numero+=1
        a=a+1
        if phrase!="":
            phraseConnecteurs=set()
            for connecteur in connecteurs:
                if " " in connecteur:
                    connecteurParties=connecteur.split(" ")
                else:
                    connecteurParties=[connecteur]
                for i in range(len(elementsPhrases[nPhrase])-len(connecteurParties)+1):
                    if connecteurParties==elementsPhrases[nPhrase][i:i+len(connecteurParties)]:
                        phraseConnecteurs.add(connecteur)
            if phraseConnecteurs:
#                print phraseConnecteurs
                api.set("connecteurs",",".join(phraseConnecteurs))
            phraseImmigration=set()
            for immigrationMot in immigrationMots:
                if " " in immigrationMot:
                    immigrationMotParties=immigrationMot.split(" ")
                else:
                    immigrationMotParties=[immigrationMot]
                for i in range(len(elementsPhrases[nPhrase])-len(immigrationMotParties)+1):
                    if immigrationMotParties==elementsPhrases[nPhrase][i:i+len(immigrationMotParties)]:
                        phraseImmigration.add(immigrationMot)
            if phraseImmigration:
#                print phraseConnecteurs
                api.set("immigration",",".join(phraseImmigration))
            api.set("nbmots",str(len(api.xpath("//tour/motBDL"))))
            api.set("id","%06d"%nPhrase)
            noeudAttachement=ligne.getparent()
            if noeudAttachement.text==None:
                noeudAttachement.tail=None
                noeudAttachement.addnext(api)
            else:
                noeudAttachement.text=None
                try:
                    noeudAttachement.append(api)
                except TypeError:
                    print phrase, noeudAttachement
                    noeudAttachement.append(api)
        else:
            ligne.getparent().tail=None
        nPhrase+=1

#Modif GB 12/04/14
- Insertion d'un set sur les nouvellesExceptions pour éviter les entrées multiples
- Ajout d'un test pour vérifier que les nouvellesExceptions sont nouvelles

#TO DO
- Ajouter un message pour dire que le résultat a été concaténé au fichier existant si c'est le cas.

In [29]:
def extraireMotsTRS(motsCorpus,phon):
    for entry in bdlexique:
        entry=entry.strip()
        p=entry.split(u';')
        if p[0].lower() in motsCorpus:
            if p[2]=="@" and not p[3] in ["N","V","J","K"]:
                p[1]+=p[2]
                p[2]=""
                if len(p)<7:
                    for i in range(len(p)+1,7):
                        p.append("")
            phon[p[0].lower()]=(p[0],p[1],p[2],p[3],p[4],p[5],p[6],p[7],p[8],p[9])
    return phon


In [30]:
#1.2.b. mettre les phrases phonémisées dans un fichier
enteteXML=[
            u'<?xml version="1.0" encoding="UTF8" standalone="yes"?>',
            u'<?xml-stylesheet type="text/xsl" href="phonemise-TRS.xsl"?>',
#            u'<!DOCTYPE Trans SYSTEM "trans-14-corpus.dtd">'
            u'<!DOCTYPE Trans SYSTEM "trans-corpus.dtd">'
          ]

#print [etree.tostring(rootTRS,pretty_print=True,encoding="utf8").decode("utf8")]
motsAbreges={}

In [31]:
def tiretsTRS(contenuTRS):
    sortie=[]
    for ligne in contenuTRS:
        m=re.search(ur'Event desc="-(\w+)" .* extent="(\w+)"',ligne)
        if m:
            ligne=ligne.replace('desc="-%s"'%m.group(1),'desc="%s"'%m.group(1)).replace('extent="%s"'%m.group(2),'extent="end"')
        m=re.search(ur'Event desc="(\w+)-" .* extent="(\w+)"',ligne)
        if m:
            ligne=ligne.replace('desc="%s-"'%m.group(1),'desc="%s"'%m.group(1)).replace('extent="%s"'%m.group(2),'extent="begin"')
        sortie.append(ligne)
    return sortie


In [32]:
def baliserTRS(nomTRS):
    with open(nomTRS,"r") as temp:
        header= temp.readlines()[0]
        s=re.search(ur'encoding="(.+)"',header)
        if s:
            TRS=codecs.open(nomTRS,"r",encoding=s.group(1)).readlines()
        else:
            TRS=open(nomTRS,"r").readlines()
    sortie=""
    fins=[]
    debs=[]
    TRS=tiretsTRS(TRS)
    for numLigne,ligne in enumerate(TRS[2:]):
        ligne=ligne.strip()
        if 0<=numLigne <=10:
            print ligne
        disfluenceGen=re.match('<Event desc="disflu" type="(noise|lexical|pronounce|language|entities)" extent="(begin|end)"/>',ligne)
        disfluenceSpec=re.match('<Event desc="([Mm]d|[Rr]ep|[Aa]uto[Cc]|[Nn]on[Ff]inie|[Mm][Cc]oup)" type="pronounce" extent="(begin|end)"/>',ligne)        
        eventAutre=re.match('<Event desc="([^"]+)" type="([^"]+)" extent="(begin|end)"/>',ligne)        
        tagTurn=re.match('<(/?)Turn.*>',ligne)        
        if disfluenceGen:
            if debug: print "disfluGen",disfluenceGen.group(2)
            if disfluenceGen.group(2)=="begin":
                sortie+=(ligne+"\n")
                sortie+=('<disfluence type="%s">'%disfluenceGen.group(1)+"\n")
                fins.append("</disfluence>")
            elif disfluenceGen.group(2)=="end":
                sortie+=("</disfluence>"+"\n")
                chaine=fins.pop()
                if chaine!="</disfluence>":
                    print "PB",chaine, "</disfluence>", numLigne
                sortie+=(ligne+"\n")
        elif disfluenceSpec:
            if debug: print "disfluSpec",disfluenceSpec.group(2)
            if disfluenceSpec.group(2)=="begin":
                sortie+=(ligne+"\n")
                sortie+=('<disfluence type="%s">'%disfluenceSpec.group(1)+"\n")
                fins.append("</disfluence>")
            elif disfluenceSpec.group(2)=="end":
                sortie+=("</disfluence>"+"\n")
                chaine=fins.pop()
                if chaine!="</disfluence>":
                    print "PB",chaine, "</disfluence>", numLigne
                sortie+=(ligne+"\n")
        elif eventAutre:
            if debug: print "Autre",eventAutre.group(3)
            descEvent="".join([l if not l in " *" else "_" for l in eventAutre.group(1)])
            typeEvent="".join([l if not l in " *" else "_" for l in eventAutre.group(2)])
            if eventAutre.group(3)=="begin":
                sortie+=(ligne+"\n")
                sortie+=('<%s desc="%s">'%(typeEvent,descEvent)+"\n")
                fins.append("</%s>"%typeEvent)
            elif eventAutre.group(3)=="end":
#                print numLigne
                if fins:
                    sortie+=("</%s>"%typeEvent+"\n")
                    chaine=fins.pop()
                else:
                    print "PB no stack to pop", typeEvent,numLigne
                if chaine!="</%s>"%typeEvent:
                    print "PB",chaine, typeEvent,numLigne
                sortie+=(ligne+"\n")
        elif tagTurn:
            if debug: print tagTurn.group(1)+"Turn"
            if tagTurn.group(1)=="/" and fins:
                lenFins=len(fins)
                for num in range(lenFins):
                    chaine=fins.pop()
                    sortie+=(chaine+"\n")
                    debs.append(chaine.replace("/",""))
            sortie+=(ligne+"\n")
            if tagTurn.group(1)=="" and debs:
                lenDebs=len(debs)
                for num in range(lenDebs):
                    chaine=debs.pop()
                    sortie+=(chaine+"\n")
                    fins.append(chaine.replace("<","</"))
        else:
            sortie+=(ligne+"\n")
        if debug and (debs or fins):
            print debs, fins
    return sortie

In [33]:
def nomTranscriptions(dossier):
    trsNoms=[f for f in glob.glob(dossier+"/*.trs") if not ("BDL2" in f or "bdl2" in f)]
    tricoNoms=[f for f in glob.glob(dossier+"/*.trico") if not ("BDL2" in f or "bdl2" in f)]
    return trsNoms+tricoNoms
    
def nomBDL2(nomTranscription):
    result=None
    if nomTranscription.endswith(u".trs"):
        result=nomTranscription.replace(u".trs","-BDL2.xml")
    elif nomTranscription.endswith(u".trico"):
        result=nomTranscription.replace(u".trico","-BDL2.xml")
    else:
        print "pb avec nomTranscription", nomTranscription
    return result

In [34]:
oldExceptions=[]
for numDossier,dossier in enumerate(listeDossiersTRS):
    if numDossier in numDossiers:
        print numDossier, dossier
        fichiersTRS=nomTranscriptions(dossier)
        print fichiersTRS
#        fichiersTRS=[f for f in fichiersTRS if not f.endswith("-BDL2.trs")]
        fichierExceptions=dossiersTRS[dossier]
        print "fichier Exception",fichierExceptions
        print
        nouvellesExceptions=[]
        boolExceptions=True
        try:
    #        print fichierExceptions
            exceptions=codecs.open(fichierExceptions,"r",encoding='utf8')
            inconnus=exceptions.readlines()
            exceptions.close()
    #        print inconnus
        except IOError:
            boolExceptions=False
        if fichier_exceptions and boolExceptions:
            oldExceptions=[]
            for entry in inconnus:
                entry=entry.strip()
                if 0: print entry
                p=entry.split(";")
                if len(p)<9:
                    for i in range(len(p)+1,10):
                        p.append(u"")
                if 0: print p
                if len(p[1])!=0:
                    phon[p[0].lower()]=(p[0],p[1],p[2],p[3],p[4],p[5],p[6],p[7],p[8])
                oldExceptions.append(p[0].lower())
        for numTRS,nomTRS in enumerate(fichiersTRS):
            print numDossier,numTRS,nomTRS
            fichierBDL=nomBDL2(nomTRS)
            if nomTRS.split("/")[-1] in sansRebalisageFichiers:
                print "SANS reBALISER"
                xmlTRS=etree.parse(nomTRS,parser)
                print "FIN parse"
            else:
                fichierTRS=baliserTRS(nomTRS)
                print "FIN reBALISER"
                xmlTRS=etree.fromstring(fichierTRS,parser)
                print "FIN fromstring"
            (motsCorpus,motsPhrases,elementsPhrases)=listerMotsCorpus(xmlTRS)
            print "FIN lister mots"
            phon=extraireMotsTRS(motsCorpus,phon)
            print "FIN extraire mots"
            traitementTRS(xmlTRS)
            print "FIN traitement"
            with codecs.open(fichierBDL, "w", encoding='utf8') as f:
                for ligne in enteteXML:
                    f.write(ligne+u"\n")
                f.write(etree.tostring(xmlTRS,pretty_print=True,encoding="utf8").decode("utf8"))
            cliBDL=fichierBDL.replace(" ","\ ")
            cliXSLT=fichierXSLT.replace(" ","\ ")
            cliPhonTRS=fichierBDL.replace(".xml",".trs").replace(" ","\ ")
            cliText="xsltproc %s %s > %s"%(cliXSLT,cliBDL,cliPhonTRS)
            print cliText
            os.system(cliText)  
        with codecs.open(fichierExceptions, "a", encoding='utf8') as f:
            print "fichierExceptions",fichierExceptions
            for n in set(nouvellesExceptions):
                print n,
                if not (n in oldExceptions): 
                    f.write(n+u";;;;;;;;;;;;")
                    f.write("\n")
            print
        oldExceptions=[]


0 /Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0 Groupes/Solsona/
['/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0 Groupes/Solsona/1Annie-80ans_Christophe-83ans.trs', '/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0 Groupes/Solsona/1Constance-58ans.trs', '/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0 Groupes/Solsona/2Annie-80ans_Christophe-83ans.trs', '/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0 Groupes/Solsona/2Constance-58ans.trs', '/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0 Groupes/Solsona/Adriane-24ans.trs', '/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0 Groupes/Solsona/Constance_Christine-58ans.trs', '/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0 Groupes/Solsona/Jacques-62ans.trs', '/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0 Groupes/Solsona/Mariano-94ans.trs', '/Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-

<Section type="report" startTime="0" endTime="149.281">
<Turn startTime="0" endTime="5.232" mode="spontaneous">
<Sync time="0"/>

</Turn>
FIN reBALISER
FIN fromstring
FIN lister mots
FIN extraire mots
FIN traitement
xsltproc /Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS-phon.xsl /Users/gilles/pCloud\ Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0\ Groupes/Solsona/Jacques-62ans-BDL2.xml > /Users/gilles/pCloud\ Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0\ Groupes/Solsona/Jacques-62ans-BDL2.trs
0 7 /Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0 Groupes/Solsona/Mariano-94ans.trs
<Trans scribe="Adele" audio_filename="Mariano-94ans" version="2" version_date="210625" xml:lang="fr">
<Speakers>
<Speaker id="spk1" name="Interviewer" check="no" type="male" dialect="native" accent="" scope="local"/>
<Speaker id="spk2" name="Mariano" check="no" type="male" dialect="nonnative" accent="" scope="local"/>
</Speakers>
<Episode program="" air_date="24/04/2021">
<Section type

FIN traitement
xsltproc /Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS-phon.xsl /Users/gilles/pCloud\ Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0\ Groupes/CordetteLefevreNaprix/Maxime-BDL2.xml > /Users/gilles/pCloud\ Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0\ Groupes/CordetteLefevreNaprix/Maxime-BDL2.trs
1 1 /Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0 Groupes/CordetteLefevreNaprix/Nathan.trs
<Trans scribe="flori" audio_filename="Nathan - 17 ans (2)" version="3" version_date="201209">
<Speakers>
<Speaker id="spk1" name="Nora" check="no" type="female" dialect="native" accent="" scope="local"/>
<Speaker id="spk2" name="Nathan " check="no" type="male" dialect="native" accent="" scope="local"/>
</Speakers>
<Episode>
<Section type="report" startTime="0" endTime="77.693">
<Turn startTime="0" endTime="0.53">
<Sync time="0"/>

</Turn>
FIN reBALISER
FIN fromstring
FIN lister mots
FIN extraire mots
FIN traitement
xsltproc /Users/gilles/ownCloud/Cours/Bordeaux/

FIN extraire mots
FIN traitement
xsltproc /Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS-phon.xsl /Users/gilles/pCloud\ Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0\ Groupes/CordetteLefevreNaprix/Stephanie-BDL2.xml > /Users/gilles/pCloud\ Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0\ Groupes/CordetteLefevreNaprix/Stephanie-BDL2.trs
1 10 /Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0 Groupes/CordetteLefevreNaprix/Thomas1.trs
<Trans scribe="flori" audio_filename="Thomas-audio-1" version="8" version_date="210111">
<Speakers>
<Speaker id="spk1" name="Thomas" check="no" type="male" dialect="native" accent="" scope="local"/>
</Speakers>
<Episode>
<Section type="report" startTime="0" endTime="108.287">
<Turn startTime="0" endTime="1.442">
<Sync time="0"/>

</Turn>
<Turn speaker="spk1" startTime="1.442" endTime="3.753">
FIN reBALISER
FIN fromstring
FIN lister mots
FIN extraire mots
FIN traitement
xsltproc /Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources

2 1 /Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0 Groupes/BadiGrousseau/colere.trs
<Trans scribe="emeli" audio_filename="use se fait griller par l'épicier, pleure et passe un sale quart d'heure HD-320k" version="4" version_date="210113">
<Speakers>
<Speaker id="spk1" name="Vendeur" check="no" type="male" dialect="native" accent="" scope="local"/>
<Speaker id="spk2" name="(no speaker)" check="no" dialect="native" accent="" scope="local"/>
<Speaker id="spk3" name="Clara" check="no" dialect="native" accent="" scope="local"/>
</Speakers>
<Episode>
<Section type="report" startTime="0" endTime="306.816">
<Turn startTime="0" endTime="0.289" speaker="spk1">
<Sync time="0"/>
police
FIN reBALISER
FIN fromstring
FIN lister mots
FIN extraire mots
FIN traitement
xsltproc /Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS-phon.xsl /Users/gilles/pCloud\ Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0\ Groupes/BadiGrousseau/colere-BDL2.xml > /Users/gilles/pCloud\ Drive/FOD/GB

on est une maison et on est pas sortis on y arrive pas
</Turn>
FIN reBALISER
FIN fromstring
FIN lister mots
FIN extraire mots
FIN traitement
xsltproc /Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS-phon.xsl /Users/gilles/pCloud\ Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0\ Groupes/BadiGrousseau/tristesse2-BDL2.xml > /Users/gilles/pCloud\ Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0\ Groupes/BadiGrousseau/tristesse2-BDL2.trs
2 10 /Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0 Groupes/BadiGrousseau/tristesse3.trs
<Trans scribe="Admin" audio_filename="charlie-hebdo-limmense-emotion-de-patrick-pelloux (2)" version="1" version_date="201204">
<Speakers>
<Speaker id="spk1" name="Patrick Pelloux" check="no" type="male" dialect="native" accent="" scope="local"/>
<Speaker id="spk2" name="Présentateur" check="no" type="male" dialect="native" accent="" scope="local"/>
</Speakers>
<Episode>
<Section type="report" startTime="0" endTime="657.842">
<Turn startTime="0" end

<Turn startTime="5.707" endTime="6.116">
FIN reBALISER
FIN fromstring
FIN lister mots
FIN extraire mots
FIN traitement
xsltproc /Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS-phon.xsl /Users/gilles/pCloud\ Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0\ Groupes/DupuyOlengaOwanlele/crise-de-la-quarantaine-comment-gerer-en-4-etapes_transcription-BDL2.xml > /Users/gilles/pCloud\ Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0\ Groupes/DupuyOlengaOwanlele/crise-de-la-quarantaine-comment-gerer-en-4-etapes_transcription-BDL2.trs
3 3 /Users/gilles/pCloud Drive/FOD/GB/Cours/L2-Corpus/Corpus-2020/0 Groupes/DupuyOlengaOwanlele/Discussion enfant de 3 ans_transcription.trs
<Trans scribe="Roselyne" audio_filename="Discussion enfant de 3 ans" version="4" version_date="210526">
<Speakers>
<Speaker id="spk1" name="enfant" check="no" type="male" dialect="native" accent="" scope="local"/>
<Speaker id="spk2" name=" " check="no" dialect="native" accent="" scope="local"/>
<Speaker id="spk3" name="p

In [35]:
numPremierDossier=25
numDernierDossier=0
if numDernierDossier<numPremierDossier or numDernierDossier==0:
    numDernierDossier=100
elif numDernierDossier==numPremierDossier:
    numDernierDossier+=1
numDossiers=range(numPremierDossier,numDernierDossier)

# Problèmes de noms de fichiers

In [36]:
motsPhrases

[[],
 [u'j',
  u'ai',
  u'fait',
  u'plusieurs',
  u'enqu\xeates',
  u'de',
  u'plusieurs',
  u'terrains',
  u'pour',
  u'mieux',
  u'comprendre',
  u'les',
  u'liens',
  u'qu',
  u'on',
  u'd\xe9veloppe',
  u'en',
  u'contexte',
  u'\xe9lectronique',
  u'les',
  u'modalit\xe9s',
  u'du',
  u'rapport',
  u'\xe0',
  u'l',
  u'autre',
  u'en',
  u'ligne'],
 [u'(h)',
  u'e',
  u'j',
  u'ai',
  u'fait',
  u'un',
  u'premier',
  u'travail',
  u'd',
  u'observation',
  u'participante',
  u'd',
  u'enqu\xeate',
  u'en',
  u'ligne',
  u'mais',
  u'aussi',
  u'euh',
  u'hors',
  u'ligne',
  u'(h)e',
  u'dans',
  u'un',
  u'canal',
  u'de',
  u'chat'],
 [u'e',
  u'd',
  u'internet',
  u'relais',
  u'chat',
  u'quelque',
  u'part',
  u'entre',
  u'e',
  u'quatre',
  u'vingt',
  u'dix',
  u'neuf',
  u'deux',
  u'mille',
  u'et',
  u'e',
  u'deux',
  u'mille',
  u'deux'],
 [u'(h)',
  u'e',
  u'dans',
  u'le',
  u'cadre',
  u'de',
  u'ma',
  u'th\xe8se',
  u'de',
  u'doctorat'],
 [u'(h)',
  u'e',
  